In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [10]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [12]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM,BatchNormalization
from keras.layers.core import Dense, Dropout

In [23]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 128

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\mchetankumar\AppData\Local\Continuum\anaconda3\envs\TaxiEnv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Users\mchetankumar\AppData\Local\Continuum\anaconda3\envs\TaxiEnv\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [16]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [24]:
# Initiliazing the sequential model
from keras import regularizers
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden,input_shape=(timesteps, input_dim),kernel_regularizer=regularizers.l2(1e-4)))
model.add(BatchNormalization())
# Adding a dropout layer
model.add(Dropout(0.2))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               70656     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 71,942
Trainable params: 71,686
Non-trainable params: 256
_________________________________________________________________


In [25]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [26]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 125s 17ms/step - loss: 0.9490 - acc: 0.5926 - val_loss: 0.7374 - val_acc: 0.6461
Epoch 2/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.6967 - acc: 0.6766 - val_loss: 0.8233 - val_acc: 0.6451
Epoch 3/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.4318 - acc: 0.8402 - val_loss: 0.7318 - val_acc: 0.7112
Epoch 4/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.4208 - acc: 0.8371 - val_loss: 0.3834 - val_acc: 0.8741
Epoch 5/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.1957 - acc: 0.9286 - val_loss: 0.2748 - val_acc: 0.9094
Epoch 6/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.1869 - acc: 0.9286 - val_loss: 0.2424 - val_acc: 0.9084
Epoch 7/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.1828 - acc: 0.9340 - val_loss: 0.2359 - val_acc: 

In [27]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  2      373       115        0                   0   
STANDING                 0       60       471        1                   0   
WALKING                  0        0         0      493                   2   
WALKING_DOWNSTAIRS       0        0         0        0                 418   
WALKING_UPSTAIRS         0        0         0        0                   4   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                            1  
WALKING_DOWNSTAIRS                 2  
WALKING_UPSTAIRS                 467  


In [28]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 7s 2ms/step


In [29]:
score

[0.21251263201785342, 0.9362063115032236]

In [3]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Architecture", "Hidden LSTM Units", "LSTM Layers", "Dropout","Test Loss","Accuracy"]

x.add_row(["1", "32","NA",0.50,0.41,0.91])
x.add_row(["2", "64","NA",0.25,0.30,0.92])
x.add_row(["3", "100","NA",0.75,0.56,0.90])
x.add_row(["4", "128","1",0.50,0.36,0.93])
x.add_row(["5", "128","2",0.75,0.45,0.91])
x.add_row(["6", "250","2",0.75,1.28,0.84])
x.add_row(["7", "180","2",0.25,0.50,0.90])
x.add_row(["8", "200","1",0.75,0.47,0.89])
x.add_row(["9", "128","1",0.2,0.21,0.94])
print(x)

+--------------+-------------------+-------------+---------+-----------+----------+
| Architecture | Hidden LSTM Units | LSTM Layers | Dropout | Test Loss | Accuracy |
+--------------+-------------------+-------------+---------+-----------+----------+
|      1       |         32        |      NA     |   0.5   |    0.41   |   0.91   |
|      2       |         64        |      NA     |   0.25  |    0.3    |   0.92   |
|      3       |        100        |      NA     |   0.75  |    0.56   |   0.9    |
|      4       |        128        |      1      |   0.5   |    0.36   |   0.93   |
|      5       |        128        |      2      |   0.75  |    0.45   |   0.91   |
|      6       |        250        |      2      |   0.75  |    1.28   |   0.84   |
|      7       |        180        |      2      |   0.25  |    0.5    |   0.9    |
|      8       |        200        |      1      |   0.75  |    0.47   |   0.89   |
|      9       |        128        |      1      |   0.2   |    0.21   |   0

- I was able to improve the accuracy from 90.09% to 93.62% and loss from 0.30 to 0.21